# Expedia Hotel Recommendations

# Setup

In [ ]:
import matplotlib
matplotlib.use('Agg')

In [ ]:
%matplotlib notebook

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# pd.set_option('display.max_columns', 30) # 27 columns of data in training set

from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import Imputer
from sklearn.externals import joblib

# Random Forest
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def make_features(data, destinations_data):    
    '''
    Extract date-time features from dataframe 'data'.
    Converts date_time, srch_ci, and srch_co fields  (if they exist) 
    into components (year, month, day, etc) and drops the original field.
    '''
    
    # Extract date-time features
    fields = ['date_time', 'srch_ci', 'srch_co']
    for field in fields:
        if field in data.keys():
            extract_datetimes(data, field)
            data = data.drop(field, axis=1)
    
    
    # merge in srch_destination_id d1-d149 fields
    data = pd.merge(data, destinations_data, on='srch_destination_id', how='left')

    # Check for nulls
    null_bool = data.isnull().sum()
    if null_bool.sum()>0:
        # Only impute columns with nulls
        has_null = data.columns[null_bool>0]

        imp = Imputer(missing_values='NaN', strategy='mean', axis=0, copy=False)
        data[has_null] = imp.fit_transform(data[has_null])
    return data

def extract_datetimes(data, field):
    data[field] = pd.to_datetime(data[field],errors='coerce')
    data[field+'_year'] = data[field].dt.year
#     data[field+'_month'] = data[field].dt.month
#     data[field+'_day'] = data[field].dt.day
#     data[field+'_hour'] = data[field].dt.hour
#     data[field+'_minute'] = data[field].dt.minute
    data[field+'_dayofyear'] = data[field].dt.dayofyear
    data[field+'_dayofweek'] = data[field].dt.dayofweek

def make_PCA(X, n_comp):
    pca = PCA(n_components=n_comp)
    pca.fit(X)
    return pca

def apk(actual, predicted, k=5):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=5):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

def mapk_score(X, y, estimator, num_splits=10):
# Score MAP@5 for X using estimator against target y, 
# splits into num_splits sets to reduce memory requirement

    n_test = X.shape[0]
    top_pred_hotel_cluster = []

    for i in range(num_splits):￼
        prob_prediction = estimator.predict_proba(X_train[int(i*n_test/num_splits):int((i+1)*n_test/num_splits)])
        top_pred_hotel_cluster.extend((np.argsort(prob_prediction)[:,-5:]).tolist())

    return mapk([[i] for i in y.values.tolist()], top_pred_hotel_cluster)

In [ ]:
use_PCA = False
save_preds = True

# Train: Random Forest Model

In [ ]:
cols = ['site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt', 'srch_children_cnt',
       'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id',
       'is_booking', 'cnt', 'hotel_country', 'hotel_market', 'hotel_continent',
       'hotel_cluster']
# 'date_time', 

cols_test = cols[:17]
cols_test.extend(cols[19:-1])

In [ ]:
destinations_data = pd.read_csv('data/destinations.csv')

chunksize = 380000
chunk_train = pd.read_csv('data/train.csv', usecols=cols, chunksize=chunksize, iterator=True)
chunk_test = pd.read_csv('data/test.csv', usecols=cols_test, chunksize=chunksize, iterator=True)

In [ ]:
# Pull a row for headers
test_data = make_features(chunk_test.get_chunk(1), destinations_data)  # 2,528,244 total lines

In [ ]:
n_estimators = 50
expedia_rfc = RandomForestClassifier(n_estimators=n_estimators, max_leaf_nodes=10, n_jobs=3)

In [ ]:
%%time 
for i in range(1,100):
    train_data = make_features(chunk_train.get_chunk(chunksize), destinations_data) # 37,670,294 total lines

    # Take features from columns in test data, ignoring some fields uniquely in train
    features = test_data.columns.tolist()[1:]

    X_all = train_data.ix[:,features]
    y_all = train_data.ix[:,'hotel_cluster']

    if use_PCA:
        X = X_all.as_matrix()
        pca = make_PCA(X, 50)
        X = pca.transform(X)
    else:
        X = X_all   

    # Generate training and cross-validation features
    X_train, X_cv, y_train, y_cv = train_test_split(X, y_all, train_size=.8, random_state=1)

    # Add more estimators
    expedia_rfc.set_params(n_estimators=i*n_estimators, warm_start=True) 
    
    # Fit the model
    expedia_rfc = expedia_rfc.fit(X_train, y_train)

    # Cross-validate the model
    num_splits=int(chunksize/1000)

    score_train = mapk_score(X_train, y_train, expedia_rfc, num_splits=num_splits)
    score_cv = mapk_score(X_cv, y_cv, expedia_rfc, num_splits=num_splits)

    print ('Iteration:', i, ' Training Score:', score_train, ' CV Score:', score_cv) 

In [ ]:
feature_importance = zip(features, expedia_rfc.feature_importances_)
for x in sorted(feature_importance, key=lambda x: -x[1]):
    print (x)

In [ ]:
joblib.dump(expedia_rfc, 'model/expedia_rfc.pkl') 

# Predict

In [ ]:
expedia_rfc = joblib.load('model/expedia_rfc.pkl')

In [ ]:
if use_PCA:
    X_test = pca.transform(X_test.as_matrix())

In [ ]:
# Pull a row for headers
test_data = make_features(chunk_test.get_chunk(1000000), destinations_data)  # 2,528,244 total lines

In [ ]:
X_test = test_data.ix[:,features]

In [ ]:
%%time

# break the test set into n = num_split sets to predict on
num_splits = 10
n_test = X_test.shape[0]
top_pred_hotel_cluster = []

for i in range(num_splits):
    prob_prediction = expedia_rfc.predict_proba(X_test[int(i*n_test/num_splits):int((i+1)*n_test/num_splits)])
    top_pred_hotel_cluster.extend([' '.join([str(hotel) for hotel in row]) for row in np.argsort(prob_prediction)[:,-5:]])

In [ ]:
submission = pd.DataFrame(top_pred_hotel_cluster, columns=['hotel_cluster'])

In [ ]:
if submission.shape[0] == 2528243:
    submission.to_csv('expedia-rf-2016-05-01-s1.csv', index_label='Id')
else:
    print('submission size does not match correct value')